In [53]:
!pip -q install fasttext
# !pip -q install datasets
# from datasets import load_dataset
import nltk
import string
import os
import numpy as np
import fasttext
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import brown
from nltk.corpus import wordnet

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [48]:
def readingfile(filename):
    with open(filename, 'r') as file:
        next(file)
        lines = file.readlines()
        # word1_list = []
        # word2_list = []
        # simlex999_list = []
        scores = {}

        for line in lines:
            data = line.strip().split('\t')
            # print(data)

            word1 = data[0]
            word2 = data[1]
            simlex999 = float(data[3])

            # word1_list.append(word1)
            # word2_list.append(word2)
            key = word1 + " " + word2
            scores[key] = simlex999
            # simlex999_list.append(simlex999)
    # for key in scores.keys():
    #     print(key, scores[key])
    return scores

def find_syns(word):
    syn = []
    ant = []
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            if '_' not in lemma.name():
                syn.append(lemma.name())
            if lemma.antonyms():
                if '_' not in lemma.antonyms()[0].name():
                    ant.append(lemma.antonyms()[0].name())

    syn = set(syn)
    ant = set(ant)
    return syn, ant

def cosine_similarity(array1, array2):
    dot_product = np.dot(array1, array2)
    norm_array1 = np.linalg.norm(array1)
    norm_array2 = np.linalg.norm(array2)
    cosine_similarity = dot_product / (norm_array1 * norm_array2)
    return cosine_similarity


In [49]:
brown_token_list = list(brown.words())
brown_token_list = [word.lower() for word in brown_token_list if not all(char in string.punctuation for char in word)]
# dataset = load_dataset("mindchain/wikitext2")
# REMOVE THIS
# brown_token_list = brown_token_list[:10000]
brown_string = " ".join(brown_token_list)

with open("data.txt", "w") as file:
    file.write(brown_string)


test_scores = readingfile("SimLex-999.txt")

In [68]:
tp, tn, fp, fn = 0, 0, 0, 0
decision_bound = 4
count = 0

if os.path.exists("model.bin"):
    model = fasttext.load_model("model.bin")
else:
    model = fasttext.train_unsupervised("data.txt")
    model.save_model("model.bin")


# oldie = model.get_word_vector("Chomsky")
# age = model.get_word_vector("Antarctica")

# print(cosine_similarity(oldie, age))

for key in test_scores.keys():
    word1, word2 = key.split()
    score = test_scores[key]
    syn1, ant1 = find_syns(word1)
    syn2, ant2 = find_syns(word2)
    if (word1 in ant2) or (word2 in ant1):
        if (score < decision_bound):
            tn += 1
        else:
            fn += 1
            # print(word1, word2, "fn")
    elif (word1 in syn2) or (word2 in syn1):
        if (score >= decision_bound):
            tp += 1
        else:
            fp += 1
            # print(word1, word2, "fp")
    else:
        vec1 = model.get_word_vector(word1)
        vec2 = model.get_word_vector(word2)
        cos_sim = cosine_similarity(vec1, vec2)
        if (cos_sim > 0.5):
            if (score >= decision_bound):
                tp += 1
            else:
                fp += 1
        else:
            if (score < decision_bound):
                tn += 1
            else:
                fn += 1


if (tp + tn + fp + fn > 0):
    print(round((tp + tn)/(fp + fn + tp + tn), 3))
    # print(tp, tn, fp, fn)

0.611
